In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import necessary libraries and packages

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets 
import torchvision.transforms as transforms

import warnings
warnings.filterwarnings('ignore')

## Create fully connected networks

In [21]:
class NN(nn.Module):
    def __init__(self, input_size,  num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x


## Set device

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

## Hyper parameters

In [23]:
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
epochs = 1

## Load the data

In [6]:
transform = transforms.ToTensor() 

# 1. Training Dataset
train_dataset = datasets.MNIST(
    root='./data', 
    train=True, 
    transform=transform, 
    download=True
)

# 2. Test Dataset
test_dataset = datasets.MNIST(
    root='./data', 
    train=False, 
    transform=transform, 
    download=True 
)

100%|██████████| 9.91M/9.91M [00:00<00:00, 15.0MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 366kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 3.39MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.41MB/s]


In [29]:
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle=True)

## Initialize the network

In [24]:
model = NN(input_size = input_size, num_classes = num_classes).to(device)

## Loss and Optimizer

In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

## Train the network

In [25]:
for epoch in range(epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device = device)
        targets = targets.to(device = device)

        # Get to correct reshape
        data = data.reshape(data.shape[0], -1)

        # Farward pass
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # Gradient Descent
        optimizer.step() 

## Check accuracy on tarining and test, to look how good the model is performing

In [27]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on testing data")
        
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device = device)
            y = y.to(device = device)

            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

In [30]:
model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 4808 / 60000 with accuracy 8.01
Checking accuracy on testing data
Got 796 / 10000 with accuracy 7.96
